# Write outlook online meeting schedule to Google Calendar using Google Calendar API
## Steps:
* From Google Calendar get all events and store them into the event list.
* From Outlook get online meeting appointments which in specific term and store them into appointment list.
* Using EntryId, find an event in the event list. If it is found, update it and delete it from the event list. If it is not found, insert it.
* Delete all events that remain in the event list. They are deleted or out of term.

In [6]:
#from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import re
import win32com.client

In [7]:
import json
# load setting file(json)
with open("calendar.json", "r", encoding="utf-8") as f:
    dic = json.load(f)

# "id":"xxxxx@group.calendar.google.com"
calendar_id = dic["id"]

# "tools": {"zoom": "zoom.us","webex": "webex.com", "teams": "teams.microsoft.com"}
tools = dic["tools"]

if "omit" in dic:
    omit=dic["omit"]
else:
    omit=[]

if "weeks" in dic:
    weeks_ahead=dic["weeks"]
else:
    weeks_ahead=3

for k in tools:
    print(k,tools[k])
print("omit",omit)
print("weeks ahead",weeks_ahead)

zoom zoom.us
webex webex.com
teams teams.microsoft.com
omit []
weeks ahead 3


In [8]:
# Outlook appointment search term:From.
st = datetime.date.today() - datetime.timedelta(days=1)
# Outlook appointment search term:To.
ed = datetime.date.today() + datetime.timedelta(weeks=weeks_ahead)

# Google settingsレンダのやつ
SCOPES = ["https://www.googleapis.com/auth/calendar"]
cred_json = "credentials.json"

# Load token
creds = None
if os.path.exists("token.pickle"):
    with open("token.pickle", "rb") as token:
        creds = pickle.load(token)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(cred_json, SCOPES)
        creds = flow.run_local_server(port=0)
    with open("token.pickle", "wb") as token:
        pickle.dump(creds, token)

# Create Google Calendar service
service = build("calendar", "v3", credentials=creds)

# Get all events from Google Calendar
print("Getting all events")
event_result = (
    service.events()
    .list(
        calendarId=calendar_id,
        #                                     timeMin=st.isoformat()+'T00:00:00Z',
        #                                     timeMax=ed.isoformat()+'T00:00:00Z',
        singleEvents=True,
        orderBy="startTime",
    )
    .execute()
)
events = event_result.get("items", [])

# Display all events
if not events:
    print("No upcoming events found.")
for i, event in enumerate(events):
    start = event["start"].get("dateTime", event["start"].get("date"))
    start = datetime.datetime.strptime(start[:-6], "%Y-%m-%dT%H:%M:%S")
    print("{:02}".format(i), start, event["summary"])

Getting all events
00 2022-04-17 00:00:00 zoom:Tet:Jenny meeting


In [5]:
# tools = {"zoom": "zoom.us", "webex": "webex.com", "teams": "teams.microsoft.com"}

# get an event that has specific entry-id from events and return its index.
def getEventIndex(events, item):
    for i, ev in enumerate(events):
        if item.EntryId in ev["description"]:
            start = ev["start"].get("dateTime", ev["start"].get("date"))
            # You need [timzone] to compare so I added .astimezone().
            # I don't know why but Outlook appointments are in UTC timezone 
            # and after .astimezone() they change to JST. So you have to change it to UTC.
            start = (
                datetime.datetime.strptime(start[:-6], "%Y-%m-%dT%H:%M:%S")
                .astimezone()
                .replace(tzinfo=datetime.timezone.utc)
            )
            td = start - item.Start

            if td == datetime.timedelta(0):
                return i
    return -1


# is the appointment onlie meeting?
def isOnline(item):
    for k, v in tools.items():
        if v in item.Body:
            for sub in omit:
                if sub in item.Subject:
                    return False, None
            return True, k
    return False, None


# objects for Outlook
app = win32com.client.Dispatch("Outlook.Application")
root = app.Session.DefaultStore.GetRootFolder()
ns = app.GetNamespace("MAPI")
cal = ns.GetDefaultFolder(9)

# filter string for specific term
filterStr = '[Start]>="{0}" and [Start]<"{1}"'.format(
    st.strftime("%Y/%m/%d"), ed.strftime("%Y/%m/%d")
)
print("filter", filterStr)

# Sort appointments and divide reccurcible ones.
appointments = cal.Items
appointments.sort("[Start]")
appointments.IncludeRecurrences = True
restricted = appointments.Restrict(filterStr)
print("from", st, "to", ed)
counter = 0

# Access to Outlook appointment
for item in restricted:
    # Read each item's body and find tools string in it. If they has, that's what you need
    ret, prefix = isOnline(item)
    countStr = "{:02}".format(counter)
    if ret:
        # Add pre-fix 
        summary = prefix + ":" + item.subject

        # Copy body part until you find password
        bd = item.body.splitlines()
        desc = ""
        for line in bd:
            desc += line + "\n"
            if "パスワード" in line:
                break
        desc += "EntryId={}".format(item.EntryId)
        # Build a JSON data for a google calendar event.
        event = {
            "summary": summary,
            "description": desc,
            "start": {
                "dateTime": item.Start.strftime("%Y-%m-%dT%H:%M:%S"),
                "timeZone": "Japan",
            },
            "end": {
                "dateTime": item.End.strftime("%Y-%m-%dT%H:%M:%S"),
                "timeZone": "Japan",
            },
        }

        # Check if the event already in envets list
        index = getEventIndex(events, item)

        if index < 0:
            # It's not found in list, so insert a new event
            print(countStr, "insert", item.Start.isoformat(), item.subject)
            service.events().insert(
                calendarId=calendar_id,
                body=event).execute()
        else:
            # It found in the list, so POP it to delete.
            ev = events.pop(index)
            updated = datetime.datetime.fromisoformat(
                ev["updated"].replace("Z", "+00:00")
            )
            modified = item.LastModificationTime + datetime.timedelta(hours=-9)
            # print(modified.isoformat(),updated)
            if updated < modified:
                # Update it
                print(countStr, "update", item.Start.isoformat(), item.subject)
                service.events().update(
                    calendarId=calendar_id, 
                    eventId=ev["id"], 
                    body=event
                ).execute()
            else:
                print(countStr, "no upd", item.Start.isoformat(), item.subject)
            start = ev["start"].get("dateTime", ev["start"].get("date"))
            start = datetime.datetime.strptime(start[:-6], "%Y-%m-%dT%H:%M:%S")
        counter += 1

# Delete all events remaining in the list
for event in events:
    print("delete", event["start"]["dateTime"], event["summary"])
    service.events().delete(
        calendarId=calendar_id, 
        eventId=event["id"]).execute()
print("done.")

filter [Start]>="2022/04/09" and [Start]<"2022/05/01"
from 2022-04-09 to 2022-05-01
00 insert 2022-04-17T00:00:00+00:00 Tet:Jenny meeting
done.
